In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/GAN/

KeyboardInterrupt: ignored

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import math
import time
import matplotlib.pyplot as plt

from make_model import make_discriminator_model,make_generator_model
from DP import differential_privacy
from help_function import flatten,reshape

In [ ]:
#Parameters for federated learning
whether_warm_up = False
warm_up_epochs = 25
warm_up_batch_size = 256
warm_up_learning_rate = 0.001
warm_up_data_size = 3000

Train_round = 300
Client_learning_rate = 1e-3
Client_batch_size = 2048

Parameter_shape = []
Parameter_number = 0
Total_data_amount = 0

Test_acc = 0 #updated after each round
#Parameters for differential privicy
Whether_differential_privacy = True
Theta_d = 1
Theta_u = 0.1
Gamma = 0.001
Tua = 0.0001
Privacy_budget_per_para = 50
#Max_select_number = 10000

#Parameters for GAN
Whether_attack = True
Fake_label = 10
Attack_label = 3
Start_attack_acc = 0.8
Epoch_per_attack = 100
Noise_dimension = 100
Generated_picture_number = 256 #生成的图像数量，用于训练GAN
Generator_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, decay=1e-7)
Discriminator_optimizer = tf.keras.optimizers.SGD(learning_rate=1e-4, decay=1e-7)

Num_examples_to_generate = 36 #生成的图像数量，用于引诱其他clients输出更多信息

Num_generate_for_show = 36 #生成的图像数量来展示

#Parameters for test


%matplotlib inline

In [ ]:
cross_entropy = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
def generator_loss(fake_output,attack_label):

  ideal_result = np.zeros(len(fake_output))
  for i in range(len(ideal_result)):
    # The class which attacker intends to get
    ideal_result[i] = attack_label
  return cross_entropy(ideal_result,fake_output)
def discriminator_loss(fake_output):    
  fake_result = np.zeros(len(fake_output))
  # Attack label
  for i in range(len(fake_result)):
    fake_result[i] = Fake_label
  return cross_entropy(fake_result, fake_output)

In [ ]:
#Clients's behavior
class Client:
  def __init__(self,id):
    self.model = make_discriminator_model()
    self.model.compile(optimizer=keras.optimizers.Adam(learning_rate=Client_learning_rate),
                       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                       metrics=['accuracy'])
    server_Weight,largest_stat_index = server.send(1)
    self.model.set_weights(reshape(server_Weight,Parameter_shape))
    self.data = None
    self.labels = None
    self.data_amount = 0
    self.id = id

    self.malicious = False
    self.attack_label = None
    self.generator = None
    self.attack_round = 0
  def training_step(self):
    Weight1 = self.model.get_weights()
    Weight2_flattened,largest_stat_index = server.send(Theta_d)
    #print(len(largest_stat_index))
    largest_stat_index.sort()
    #print(largest_stat_index[0:20])
    Weight1_flattened = flatten(Weight1)
    #print(Weight1_flattened[0:20])
    #print(Weight2_flattened[0:20])
    Weight1_flattened[largest_stat_index] = Weight2_flattened[largest_stat_index]
    #print(Weight1_flattened[0:20])
    Weight1 = reshape(Weight1_flattened,Parameter_shape)
    self.model.set_weights(Weight1)

    #print(Weight1_flattened)
    if(self.malicious==True and Test_acc>Start_attack_acc and self.model.evaluate(test_images,test_labels,verbose=0)[1] > Start_attack_acc):
      self.attack()
    
    self.model.fit(self.data,self.labels,validation_split=0,epochs=1,batch_size=Client_batch_size,verbose=1)
    Gradient = np.array(self.model.get_weights()) - np.array(Weight1)
    #start = time.time()
    if Whether_differential_privacy == True:
      Gradient = differential_privacy(Gradient,Privacy_budget_per_para,Gamma,Theta_u,Tua,Parameter_shape)
      #print(flatten(Gradient)[0:20])
    #print("Time spend on DP:",time.time()-start)
    return self.data_amount,Gradient
  def attack(self):
    print("Attack round ",self.attack_round,"begins.")
    self.attack_round = self.attack_round + 1
    for i in range(Epoch_per_attack):
      noise = tf.random.normal([Generated_picture_number,Noise_dimension])
      with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = self.generator(noise, training=True)
        fake_output = self.model(generated_images, training=False)
        gen_loss = generator_loss(fake_output,self.attack_label)
        disc_loss = discriminator_loss(fake_output)
      Gradient_of_generator = gen_tape.gradient(gen_loss,self.generator.trainable_variables)
      Generator_optimizer.apply_gradients(zip(Gradient_of_generator,self.generator.trainable_variables))
      Gradients_of_discriminator = disc_tape.gradient(disc_loss, self.model.trainable_variables)
      Discriminator_optimizer.apply_gradients(zip(Gradients_of_discriminator, self.model.trainable_variables))
    noise = tf.random.normal([Num_examples_to_generate, Noise_dimension])
    #将之前生成的图片丢弃
    generated_Image = self.generator(noise,training = False)
    malicious_Image = np.array(generated_Image)
    malicious_Label = np.array([self.labels[10]]*Num_examples_to_generate)
    self.data = self.data[0:self.data_amount]
    self.labels = self.labels[0:self.data_amount]
    np.append(self.data,malicious_Image)
    np.append(self.labels,malicious_Label)
  def generate_image(self,round):
    if self.malicious == False:
      assert 0,"I am innocent and would not generate any images"
    noise = tf.random.normal([Num_generate_for_show, Noise_dimension])
    generated_Image = self.generator(noise,training = False)
    fig = plt.figure(figsize=(6,6))
    for i in range(Num_generate_for_show):
      plt.subplot(6, 6, i+1)
      plt.imshow(generated_Image[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')
    plt.savefig('/content/gdrive/MyDrive/GAN/image/image_at_round_{:04d}.png'.format(round))
    plt.show()

In [ ]:
# Server's behavior
class Server:
  def __init__(self):
    self.model = make_discriminator_model()
    self.model.compile(optimizer=keras.optimizers.Adam(learning_rate=warm_up_learning_rate),
                       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                       metrics=['accuracy'])
    self.warm_up_data = None
    self.warm_up_labels = None
    self.warm_up_data_amount = 0
    self.stat = []
    for i in range(Parameter_number):
      self.stat.append([i,0])

  def warm_up(self):
    assert self.warm_up_data_amount != 0,"There is no data for warm up!"
    self.model.fit(self.warm_up_data,self.warm_up_labels,validation_split=0,epochs=warm_up_epochs,batch_size=warm_up_batch_size,verbose=0)

  #A participant uploads gradients
  def new_gradient(self,Gradient):
    Weight = self.model.get_weights()
    Weight += Gradient
    Gradient_flattened = flatten(Gradient)
    for i in range(len(Gradient_flattened)):
      if(Gradient_flattened[i] != 0):
        self.stat[i][1] = self.stat[i][1]+1
    self.model.set_weights(Weight)
    
  #A participant downloads parameters.
  def send(self,theta):
    largest_stat_index = [i[0] for i in sorted(self.stat,key = lambda x:(x[1],x[0]),reverse=True)[0:math.floor(theta*Parameter_number)]]
    Weight=self.model.get_weights()
    Weight_flattened = flatten(Weight)

    selected_Weight = np.zeros(Parameter_number)
    for i in largest_stat_index:
      selected_Weight[i] = Weight_flattened[i]
    return selected_Weight,largest_stat_index

In [ ]:
#Federated learning preparation
#get Parameter number and shape
test_model = make_discriminator_model()
test_weight = test_model.get_weights()
for w in test_weight:
  shape = w.shape
  p_num = 1
  for i in shape:
    p_num = p_num*i
  Parameter_shape.append((p_num,w.shape))
  Parameter_number = Parameter_number + p_num 
del test_model,test_weight

#load data
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5   # Normalization
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1).astype('float32')
test_images = (test_images - 127.5) / 127.5   # Normalization
Total_data_amount = len(train_labels)
###shuffle the data to get the warm up data
state = np.random.get_state()
np.random.shuffle(train_images)
np.random.set_state(state)
np.random.shuffle(train_labels)
warm_up_data = train_images[0:warm_up_data_size]
warm_up_labels = train_labels[0:warm_up_data_size]

#init server and clients
server = Server()
server.warm_up_data = warm_up_data
server.warm_up_labels = warm_up_labels
server.warm_up_data_amount = warm_up_data_size
if whether_warm_up == True:
  server.warm_up()
clients = []
for i in range(10):
  client = Client(i)
  client.data = train_images[train_labels==i]
  client.labels = train_labels[train_labels==i]
  client.data_amount = len(client.data)

  state = np.random.get_state()
  np.random.shuffle(client.data)
  np.random.set_state(state)
  np.random.shuffle(client.labels)
  clients.append(client)
del train_images,train_labels

if Whether_attack == True:
  clients[0].malicious = True
  clients[0].attack_label = 3
  clients[0].generator = make_generator_model()

In [ ]:
total_start = time.time()
for i in range(Train_round):
  start = time.time()
  # Round robin
  for j in range(10):
    data_amount,Gradient = clients[j].training_step()
    Gradient = np.array(Gradient) * data_amount/Total_data_amount
    server.new_gradient(Gradient)
  # Gradient_sum = np.zeros(Parameter_number)
  # for j in range(10):
  #   data_amount,Gradient = clients[j].training_step()
  #   Gradient = flatten(Gradient) * data_amount/Total_data_amount
  #   Gradient_sum = Gradient_sum + Gradient
  # server.new_gradient(np.array(reshape(Gradient_sum,Parameter_shape)))
  clients[0].generate_image(i)
  end = time.time()
  print("Time for round:",i,"is ",end-start)
  test_loss, Test_acc = server.model.evaluate(test_images,test_labels,verbose=0)
  print("test_loss:",test_loss,"test_acc:",Test_acc)
print("total time:",time.time()-total_start) 